### Import thư viên

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

### Đọc dữ liệu MIMIC-III

In [49]:
# Bước 1: Load dữ liệu từ MIMIC-III
diagnoses = pd.read_csv('DIAGNOSES_ICD.csv')
prescriptions = pd.read_csv('PRESCRIPTIONS.csv')
icd_names = pd.read_csv('D_ICD_DIAGNOSES.csv')

In [50]:
unique_hadm_count = prescriptions['hadm_id'].nunique()
print("Số lượng hadm_id duy nhất:", unique_hadm_count)

Số lượng hadm_id duy nhất: 122


### Gom nhóm dữ liệu theo Hdam_id (mã lần nhập viện)

In [51]:

# Bước 2: Map mã ICD sang tên bệnh 
diagnoses = diagnoses.merge(icd_names[['icd9_code', 'short_title']],
                            left_on='icd9_code', right_on='icd9_code', how='left')
diagnoses['diagnosis'] = diagnoses['short_title']

# Bước 3: Gom nhóm theo hadm_id
grouped = {}

# Thêm chẩn đoán
for hadm, group in diagnoses.groupby('hadm_id'):
    grouped.setdefault(hadm, []).extend(['diag_' + d for d in group['diagnosis'].dropna().unique()])

# Thêm thuốc (gộp theo drug_name_generic để loại bỏ các lần kê trùng)

for hadm, group in prescriptions.groupby('hadm_id'):
    # Lấy danh sách thuốc generic duy nhất
    unique_drugs = group['drug_name_generic'].dropna().unique()
    grouped.setdefault(hadm, []).extend(['drug_' + d for d in unique_drugs])
    
# XÓA hadm_id = 186361 TRƯỚC KHI GHI FILE
grouped.pop(186361, None)  # dùng số nguyên, an toàn nếu không tồn tạ

# Convert transactions dict to list of rows for export
transactions = []
for hadm_id, items in grouped.items():
    transactions.append({'hadm_id': hadm_id, 'items': ';'.join(items)})

# Ghi ra file CSV
transactions_df = pd.DataFrame(transactions)
transactions_df.to_csv('transactions_diagnosis_drug.csv', index=False)


### Tạo các tập phổ biến giữa chẩn đoán và thuốc

In [53]:

# Bước 4: Chuyển sang danh sách các set để dùng cho luật kết hợp
transaction_lists = [row['items'].split(';') for row in transactions]

# (Tuỳ chọn) Lấy mẫu nhỏ nếu dữ liệu quá lớn để tránh MemoryError
sample_size = 100  # bạn có thể điều chỉnh hoặc bỏ dòng này nếu dữ liệu nhỏ
transaction_lists_sample = transaction_lists[:sample_size]

all_items = set(item for trans in transaction_lists_sample for item in trans)
print("Số lượng item duy nhất (unique items):", len(all_items))

# Mã hóa transactions sang ma trận one-hot
te = TransactionEncoder()
df_encoded = pd.DataFrame(te.fit_transform(transaction_lists_sample), columns=te.columns_)

# Bước 5: Tìm tập phổ biến dùng FP-Growth thay cho Apriori
freq_items = fpgrowth(df_encoded, min_support=0.1, use_colnames=True)

# Tạo luật kết hợp
rules = association_rules(freq_items, metric="confidence", min_threshold=0.3)

# Bước 6: Lọc luật "chẩn đoán --> thuốc"
def is_diag(items):
    return all(i.startswith('diag_') for i in items)

def is_drug(items):
    return all(i.startswith('drug_') for i in items)

diag_to_drug_rules = rules[
    rules['antecedents'].apply(is_diag) &
    rules['consequents'].apply(is_drug)
]

# Xuất luật ra file CSV
diag_to_drug_rules.to_csv('rules_diag_to_drug_fp_growth.csv', index=False)

# In một số luật mẫu
print(diag_to_drug_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head())

Số lượng item duy nhất (unique items): 866
         antecedents             consequents  support  confidence      lift
4555  (diag_CHF NOS)   (drug_Heparin Sodium)     0.30    0.857143  1.318681
4557  (diag_CHF NOS)  (drug_Docusate Sodium)     0.25    0.714286  1.400560
4558  (diag_CHF NOS)            (drug_Senna)     0.23    0.657143  1.194805
4561  (diag_CHF NOS)    (drug_Acetaminophen)     0.23    0.657143  1.059908
4562  (diag_CHF NOS)     (drug_Pantoprazole)     0.21    0.600000  1.463415


### Tạo các tập phổ biến giữa các chẩn đoán

In [54]:
# 2. Gom nhóm mã chẩn đoán theo từng lần nhập viện (HADM_ID)
transactions = diagnoses.groupby('hadm_id')['icd9_code'].apply(list).tolist()

# 3. Chuyển đổi dữ liệu sang dạng ma trận True/False
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_trans = pd.DataFrame(te_ary, columns=te.columns_)

# 4. Tìm tập phổ biến với min_support=0.01 (ví dụ)
frequent_itemsets = fpgrowth(df_trans, min_support=0.05, use_colnames=True)

# 5. Sinh luật kết hợp với min_threshold confidence=0.6
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

# 6. Chuyển frozenset thành string để dễ đọc và lưu CSV
rules['antecedents'] = rules['antecedents'].apply(lambda x: ', '.join(list(x)))
rules['consequents'] = rules['consequents'].apply(lambda x: ', '.join(list(x)))

# 7. Lưu kết quả vào file CSV
rules.to_csv('association_rules_fp_growth_output.csv', index=False)

print("Đã lưu kết quả luật kết hợp FP-Growth vào file 'association_rules_fp_growth_output.csv'")


Đã lưu kết quả luật kết hợp FP-Growth vào file 'association_rules_fp_growth_output.csv'
